In [1]:
# Cài đặt lại protobuf để sửa lỗi xung đột với TensorFlow trên Kaggle
!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tens

In [2]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import gc 

BASE_PATH = '/kaggle/input/preprocessed-hong-kong-pollutant-dataset/processed/xgboost'
TARGET_COL = 'PM2_5_log_scaled'  

In [3]:
def load_data_from_folder(base_path, split_name):
    path_pattern = os.path.join(base_path, split_name, "part-*")
    all_files = glob.glob(path_pattern)
    
    # Lọc file hợp lệ
    data_files = [f for f in all_files if "part-" in f and not f.endswith(".crc")]
    data_files.sort()
    
    print(f"--> Đang xử lý {len(data_files)} file trong '{split_name}'...")
    
    df_list = []
    for f in data_files:
        try:
            # Ưu tiên đọc parquet
            if f.endswith('.parquet'):
                df = pd.read_parquet(f)
            else:
                df = pd.read_csv(f)
            df_list.append(df)
        except Exception as e:
            print(f"  ❌ Lỗi file {os.path.basename(f)}: {e}")

    if not df_list:
        print("⚠️ Không tìm thấy dữ liệu!")
        return None
        
    return pd.concat(df_list, ignore_index=True)

In [4]:
def prepare_tabular_data(df, target_col):
    """
    Chuẩn bị dữ liệu tabular cho XGBoost.
    Input: DataFrame với các cột feature và target.
    Output X: (Samples, Features) 
    Output y: (Samples,)
    """
    # 1. Lấy nhãn y (Float32 để tiết kiệm RAM)
    y = df[target_col].values.astype(np.float32)
    
    # 2. Tìm các cột feature (loại trừ target và các cột không cần thiết)
    exclude_cols = [target_col, 'location_id', 'datetime']
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    
    print(f"Số lượng features: {len(feature_cols)}")
    print(f"Target column: {target_col}")
    
    # 3. Lấy features X
    X = df[feature_cols].values.astype(np.float32)
    
    return X, y, len(feature_cols)

In [5]:
# 1. Load Train & Val
print("--- STEP 1: LOADING DATA ---")
df_train = load_data_from_folder(BASE_PATH, 'train')
df_val   = load_data_from_folder(BASE_PATH, 'val')

# 2. Chuẩn bị dữ liệu tabular cho XGBoost
print("\n--- STEP 2: PREPARING TABULAR DATA ---")
X_train, y_train, n_feats = prepare_tabular_data(df_train, TARGET_COL)
X_val, y_val, _           = prepare_tabular_data(df_val, TARGET_COL)

print(f"Input Shape XGBoost: {X_train.shape}")
print(f"Target Shape: {y_train.shape}")
print(f"Validation Shape: {X_val.shape}")

# 3. Xóa DataFrame gốc để giải phóng RAM
del df_train, df_val
gc.collect()

--- STEP 1: LOADING DATA ---
--> Đang xử lý 7 file trong 'train'...
--> Đang xử lý 7 file trong 'val'...

--- STEP 2: PREPARING TABULAR DATA ---
Số lượng features: 71
Target column: PM2_5_log_scaled
Số lượng features: 71
Target column: PM2_5_log_scaled
Input Shape XGBoost: (201818, 71)
Target Shape: (201818,)
Validation Shape: (43061, 71)


0

In [6]:
# Cấu hình XGBoost
# tree_method='hist': Tối ưu cực nhanh cho dữ liệu lớn (tương tự LightGBM)
# device='cuda': Nếu bạn bật GPU thì bỏ comment dòng này để chạy nhanh hơn
model = XGBRegressor(
    n_estimators=2000,      # Số lượng cây tối đa
    learning_rate=0.05,     # Tốc độ học
    max_depth=8,            # Độ sâu của cây (đừng để quá sâu dễ overfitting)
    subsample=0.8,          # Chỉ dùng 80% dữ liệu mỗi lần train (tránh overfit)
    colsample_bytree=0.8,   # Chỉ dùng 80% features mỗi cây
    n_jobs=-1,              # Dùng tất cả CPU core
    tree_method='hist',     # Bắt buộc cho dữ liệu lớn
    early_stopping_rounds=50, # Dừng nếu không cải thiện sau 50 vòng
    random_state=42
    # device='cuda'         # <--- Bỏ comment nếu bật GPU T4 x2
)

print("\n--- STEP 3: TRAINING XGBOOST ---")
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=100  # In log mỗi 100 vòng
)

# Sau khi train xong, xóa X_train, X_val để lấy chỗ cho Test
del X_train, y_train, X_val, y_val
gc.collect()
print("🧹 Đã dọn dẹp RAM sau khi train.")


--- STEP 3: TRAINING XGBOOST ---
[0]	validation_0-rmse:0.12379	validation_1-rmse:0.11898
[100]	validation_0-rmse:0.02627	validation_1-rmse:0.02940
[200]	validation_0-rmse:0.02406	validation_1-rmse:0.02906
[300]	validation_0-rmse:0.02269	validation_1-rmse:0.02900
[400]	validation_0-rmse:0.02155	validation_1-rmse:0.02898
[500]	validation_0-rmse:0.02050	validation_1-rmse:0.02897
[518]	validation_0-rmse:0.02035	validation_1-rmse:0.02898
🧹 Đã dọn dẹp RAM sau khi train.


In [7]:
# 1. Load Test
print("\n--- STEP 4: TESTING ---")
df_test = load_data_from_folder(BASE_PATH, 'test')

# 2. Chuẩn bị Test data cho XGBoost
# Lưu ý: Đảm bảo prepare_tabular_data khớp với cách xử lý lúc train
X_test, y_test, _ = prepare_tabular_data(df_test, TARGET_COL)

# Xóa ngay dataframe gốc để tiết kiệm RAM
del df_test
gc.collect()

# 3. Dự báo
print("Đang dự báo...")
y_pred = model.predict(X_test)

# 4. Hàm đánh giá (Đã sửa đổi)
def evaluate(y_true, y_pred, name):
    r2 = r2_score(y_true, y_pred)
    
    # Tính MSE
    mse = mean_squared_error(y_true, y_pred)
    
    # Tính RMSE (Lấy căn bậc hai của MSE)
    rmse = np.sqrt(mse)
    
    mae = mean_absolute_error(y_true, y_pred)
    
    # Đã xóa phần tính MRE
    
    print(f"\n=== KẾT QUẢ {name} ===")
    print(f"R2 Score: {r2:.4f}")
    print(f"MSE:      {mse:.4f}")  # Hiển thị MSE
    print(f"RMSE:     {rmse:.4f}")
    print(f"MAE:      {mae:.4f}")

# Chạy đánh giá
evaluate(y_test, y_pred, "XGBoost Final Result")


--- STEP 4: TESTING ---
--> Đang xử lý 7 file trong 'test'...
Số lượng features: 71
Target column: PM2_5_log_scaled
Đang dự báo...

=== KẾT QUẢ XGBoost Final Result ===
R2 Score: 0.9553
MSE:      0.0009
RMSE:     0.0298
MAE:      0.0187


In [8]:
# Lưu mô hình XGBoost vào file .json
model_name = 'xgboost_pm25.json' # Đổi đuôi file thành .json

# Dùng hàm save_model của XGBoost
model.save_model(model_name) 

print(f"✅ Đã lưu mô hình thành công vào file: {model_name}")
print("Bạn có thể tìm thấy nó trong mục Output bên phải để tải về.")

✅ Đã lưu mô hình thành công vào file: xgboost_pm25.json
Bạn có thể tìm thấy nó trong mục Output bên phải để tải về.
